In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 22.2 MB/s eta 0:00:00
Found existing installation: transformers 4.57.0
Uninstalling transformers-4.57.0:
  Successfully uninstalled transformers-4.57.0
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 84.1 MB/s et

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import json
import pandas as pd

data_judg = []
with open("train_judg.jsonl", 'r', encoding="utf-8") as f:
    for line in f:
        data_judg.append(json.loads(line))

df_judg = pd.DataFrame(data_judg)
df_judg

,ID,Judgment
0,id_10,Case :- WRIT - C No. - 11383 of 2023\nPetition...
1,id_1000,Versus\nAppearance:\nand\nDate : 22/09/2023\n1...
2,id_1001,Non-Reportable\nCriminal Appeal No._________ o...
3,id_1002,---- Appellant\nVersus\nUmesh Sharma S/o Late...
4,id_1003,1Whether Reporters of Local Papers may be allo...
...,...,...
1195,id_995,(arising out of Special Leave Petition ( Crimi...
1196,id_996,% Reserved on: 15.03.2023\nPronounced on: 16.0...
1197,id_997,Reportable\nWrit Petition (C) No 961 of 2021\n...
1198,id_998,M/s. Biovet Private Limited ….Applicants\nIn t...


In [ ]:
data_sum = []
with open("train_ref_summ.jsonl", 'r', encoding="utf-8") as f:
    for line in f:
        data_sum.append(json.loads(line))

df_sum = pd.DataFrame(data_sum)
df_sum

,ID,Summary
0,id_10,The Allahabad High Court on Thursday stayed th...
1,id_1000,A convict in Gujarat who had secured bail in 2...
2,id_1001,A police officer failing in their fundamental ...
3,id_1002,The Chhattisgarh High Court recently observed ...
4,id_1003,The Gujarat High Court recently quashed a Firs...
...,...,...
1195,id_995,Facts sourced from a statement made by accused...
1196,id_996,The Delhi High Court recently directed mediati...
1197,id_997,The Supreme Court on Friday passed an interim ...
1198,id_998,The Bombay High Court on Thursday permitted Bi...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("akhilm97/pegasus_indian_legal")
model = AutoModelForSeq2SeqLM.from_pretrained("akhilm97/pegasus_indian_legal")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
sample=df_judg['Judgment'][5]
len(sample.split())

6719

In [ ]:
tokenized_sent=tokenizer(sample, return_tensors="pt")
print(tokenized_sent)

Token indices sequence length is longer than the specified maximum sequence length for this model (8494 > 1024). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[27782,   841,   566,  ...,   387,   107,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


In [ ]:
data = pd.merge(df_judg, df_sum[['ID','Summary']], on = 'ID', how = "left")

In [ ]:
data

,ID,Judgment,Summary
0,id_10,Case :- WRIT - C No. - 11383 of 2023\nPetition...,The Allahabad High Court on Thursday stayed th...
1,id_1000,Versus\nAppearance:\nand\nDate : 22/09/2023\n1...,A convict in Gujarat who had secured bail in 2...
2,id_1001,Non-Reportable\nCriminal Appeal No._________ o...,A police officer failing in their fundamental ...
3,id_1002,---- Appellant\nVersus\nUmesh Sharma S/o Late...,The Chhattisgarh High Court recently observed ...
4,id_1003,1Whether Reporters of Local Papers may be allo...,The Gujarat High Court recently quashed a Firs...
...,...,...,...
1195,id_995,(arising out of Special Leave Petition ( Crimi...,Facts sourced from a statement made by accused...
1196,id_996,% Reserved on: 15.03.2023\nPronounced on: 16.0...,The Delhi High Court recently directed mediati...
1197,id_997,Reportable\nWrit Petition (C) No 961 of 2021\n...,The Supreme Court on Friday passed an interim ...
1198,id_998,M/s. Biovet Private Limited ….Applicants\nIn t...,The Bombay High Court on Thursday permitted Bi...


In [ ]:
from datasets import Dataset

# Convert your merged DataFrame to HuggingFace Dataset
dataset = Dataset.from_pandas(data[['Judgment', 'Summary']])

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(
        example_batch['Judgment'],
        max_length=1024,
        truncation=True,
        padding='max_length'
    )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(
            example_batch['Summary'],
            max_length=768,
            truncation=True,
            padding='max_length'
        )

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset

Dataset({
    features: ['Judgment', 'Summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1200
})

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
import evaluate
rouge = evaluate.load("rouge")

In [ ]:
import numpy as np
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Clean up text (remove extra whitespace)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Calculate ROUGE scores
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # Add generation length statistics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["gen_len_std"] = np.std(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="text_summarization_model",
    eval_strategy="no",  # Changed since no eval dataset
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    report_to=None,
    fp16=True,
    push_to_hub=False,
    logging_steps=100,
    save_steps=500,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,  # Your single tokenized dataset
    eval_dataset=None,      # No validation set
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    compute_metrics=None,   # Remove since no evaluation
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3957908879.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss
100,3.509400
200,2.949600
300,2.512100
400,2.287900
500,2.162500
600,2.152800
700,1.995300
800,2.109800
900,2.149600
1000,2.038900


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1800, training_loss=2.2290857103135853, metrics={'train_runtime': 2384.1225, 'train_samples_per_second': 1.51, 'train_steps_per_second': 0.755, 'total_flos': 1.04020718911488e+16, 'train_loss': 2.2290857103135853, 'epoch': 3.0})

In [ ]:
df_val = pd.read_json("val_judg.jsonl", lines = True, encoding = "utf-8")
df_val

,ID,Judgment
0,id_100,"Through: Mr. Sanjeev Poddar, Sr. Advocate\nwit..."
1,id_1010,Judgment reserved on 17.11.2023\nJudgment deli...
2,id_1019,Indian Performing Right Society Limited …Appli...
3,id_1024,Reportable\nWrit Petition (C) No. 493 of 2022\...
4,id_1029,of 2023\nPetitioner :- Rajat Bajpai\nResponden...
...,...,...
195,id_944,"Shailaja\n1.Sonu Sood ]\nAged 48 years, Occupa..."
196,id_965,(a) Consideration as to whether Tiger\nSafaris...
197,id_978,Reportable\nCriminal Appeal Nos. 101-102 of 20...
198,id_99,..... Plaintiffs\nThrough: Mr. Dushyant K. Mah...


In [ ]:
df_val.head()

,ID,Judgment
0,id_100,"Through: Mr. Sanjeev Poddar, Sr. Advocate\nwit..."
1,id_1010,Judgment reserved on 17.11.2023\nJudgment deli...
2,id_1019,Indian Performing Right Society Limited …Appli...
3,id_1024,Reportable\nWrit Petition (C) No. 493 of 2022\...
4,id_1029,of 2023\nPetitioner :- Rajat Bajpai\nResponden...


In [ ]:
df_val['Judgment'].iloc[0][:200]

'Through: Mr. Sanjeev Poddar, Sr. Advocate\nwith Ms. Samiksha, Advocate.\nversus\nThrough: Mr. Sameer Vashisht, Additional\nStanding Counsel with Ms. Sanjana\nNangia, Advocate for respondent\nNos. 1 & 2.\nMr.'

In [ ]:
df_val = df_val.dropna(subset=['Judgment'])
df_val['Judgment'] = df_val['Judgment'].astype(str)

In [ ]:
val_dataset = Dataset.from_pandas(df_val[['Judgment']])

In [ ]:
def tokenize_validation(example_batch):
  input_encodings = tokenizer(example_batch['Judgment'], max_length=768, truncation=True, padding='max_length')
  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask']
  }

In [ ]:
tokenized_val = val_dataset.map(tokenize_validation, remove_columns=['Judgment'], batched = True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
tokenized_val.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask']
)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
val_dataloader = DataLoader(tokenized_val, batch_size=6, collate_fn=seq2seq_data_collator)

In [ ]:
model.eval()

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [ ]:
gen_summaries = []

In [ ]:
print("Generating summaries...")
with torch.no_grad():
  for i, batch in enumerate(val_dataloader):
    batch = {k: v.to(model.device) for k, v in batch.items() if k in ['input_ids', 'attention_mask']}

    generated_ids = model.generate(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        max_length=768,
        min_length=640,
        num_beams=4,
        length_penalty=1.5,
        early_stopping=True
    )

    summaries = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    gen_summaries.extend(summaries)

    if i % 10 == 0:
      print(f"Processed {len(gen_summaries)} samples...")

Generating summaries...
Processed 6 samples...
Processed 66 samples...
Processed 126 samples...
Processed 186 samples...


In [ ]:
import json

In [ ]:
# 10. Add summaries to DataFrame and save
df_val['Summary'] = gen_summaries

# Save to JSONL
output_file = 'answer.jsonl'
with open(output_file, 'w') as f:
    for _, row in df_val.iterrows():
        json_line = {
            'ID': row['ID'],
            'summary': row['Summary']
        }
        f.write(json.dumps(json_line) + '\n')

print(f"\nSummaries saved to {output_file}")


Summaries saved to answer.jsonl


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive"

 Ayaan_Resume.gdoc
'Colab Notebooks'
'late_submission form.gdoc'
'Theoretical Computer Science research areas:.gdoc'
'Untitled document (1).gdoc'
'Untitled document.gdoc'


In [ ]:
path = "/content/gdrive/MyDrive/text_summarization_model "
torch.save(model.state_dict(), path)

In [ ]:
import shutil

In [ ]:
shutil.copytree('/content/text_summarization_model', '/content/gdrive/MyDrive/pegasus_indlegal')

'/content/gdrive/MyDrive/pegasus_indlegal'